In [39]:
from W_utility.log import * 
from W_utility.file import * 

from NLP.porter2 import *
from NLP.word import *
from NLP.sentence import *
from NLP.sentence_keywords import * 

In [40]:
import re, math, csv
import W_utility.file as ufile
from NLP import sentence
from NLP import sentence_keywords

In [41]:
# Declare vars to empty
greater, greater_equal, greater_equal2, lower, lower_equal, lower_equal2, equal, between, selects, connect, features, temporal, temporal_con, error1, error2, symbols, numbers, unit_special, unit_ori, unit_ori_s, unit_exp, negation = "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", ""

def init_features ():
    # Import csv data as dictionary 
    feature_set = ufile.read_csv_as_dict ('data\\numeric_features.csv', 0, 1, True)
    
    # Declare Global variables from the imported data
    global greater, greater_equal, greater_equal2, lower, lower_equal, lower_equal2, equal, between, selects, connect, features, temporal, temporal_con, error1, error2, symbols, numbers, unit_special, unit_ori, unit_ori_s, unit_exp, negation
    greater, greater_equal, greater_equal2, lower, lower_equal, lower_equal2, equal, between, selects, connect, features, temporal, temporal_con, error1, error2, symbols, numbers, unit_special, unit_ori, unit_ori_s, unit_exp, negation = \
    feature_set["greater"], feature_set["greater_equal"], feature_set["greater_equal2"], feature_set["lower"], feature_set["lower_equal"], feature_set["lower_equal2"], feature_set["equal"], feature_set["between"], feature_set["selects"], feature_set["connect"], feature_set["features"], feature_set["temporal"], feature_set["temporal_con"], feature_set["error1"], feature_set["error2"], feature_set["symbols"], feature_set["numbers"], feature_set["unit_special"], feature_set["unit_ori"], feature_set["unit_ori_s"], feature_set["unit_exp"], feature_set["negation"]
    
    # ADD plural form of words in temporal 
    temporal = temporal + '|' + temporal.replace('|', 's|') + 's'
    
    # Create a new varibale that consists of words in unit_ori, unit_ori_s, plural form of words in unit_ori_s, temporal
    unit = (unit_ori + "|" + unit_ori_s.replace("|", "s|") + "s|" + unit_ori_s + "|" + temporal)
    return ""

In [42]:
init_features()

''

In [43]:
# Text replace sor sub
def preprocessing (text):
    # handle special characters
    text = text.encode().decode('ascii', 'ignore')
    
    # Strip() is used to remove whitespace. #Strip is used to remove leading & trailing characters. If omitted or None, the chars argument defaults to removing whitespace.
    # '#' is used the 'example data diabetes_Type 1.csv' to end a sentence. Hence \n\n is replaced with #
    text = text.strip().replace('\n\n', '#') 
    
    # \n is replced with empty
    text = text.replace ('\n', '')
    
    text = text.replace(u'＝','=').replace(u'＞', '>').replace(u'＜','<').replace(u'≤','<=').replace (u'≥','>=').replace(u'≦','<=').replace(u'≧','>=').replace(u'mm³','mm^3').replace(u'µl','ul').replace(u'µL','ul').replace(u'·','').replace(u'‐','-').replace(u'—','-')
    
    # Double brackets are replced with single brackets
    text = text.replace('((', '(').replace('))', ')')
    
    
    # "105&200" converted to "105 - 200"
    # "105 & 200" converted to "105 - 200"
    # & or / or ~ is replced with -
    # \1 corresponds to 1st char/string i.e (\d+) 
    # \5 corresponds to 5th char/string i.e (\d+)
    # ( |) - corresponds to if space is present or not
    text = re.sub('(\d+)( |)(~|/|&|\|)( |)(\d+)',r'\1 - \5',text) # e.g., '10~20' to '10 ~ 20'
    
    # Commas in only thousands is replaced (Makes sense as if r"(\d+),(\d+)" is used instead, July24,2017 will be changed to July242017 & in 10/19,10/20 will be changed to 10/1910/20)
    text = re.sub(r"(\d+),(\d{3})", r'\1\2', text) # 10,123 to 10123
    
    # Should be run only using DateTime function
    # If only 1 or 2 digits are present after 'digits followed by comma' 
    # Not needed to run this 
    text = re.sub(r"(\d+),(\d{1,2})", r'\1.\2', text) # 10,1 to 10.1
    
    # r"between (\d+), (\d{1,2}) (and|or) ", r'between \1.\2 \3 ' MODIFIED TO  r"between (\d+),( |)(\d{1,2}) (and|or) ", r'between \1.\3 \4 '
    # Not needed to run this
    text = re.sub(r"between (\d+),( |)(\d{1,2}) (and|or) ", r'between \1.\3 \4 ', text) # 'between 7, 5 and ' to 'between 7.5 and '
    
    # Within a text if there double spaces - replace it with a single space
    while '  ' in text:
        text = text.replace('  ',' ')
    
    # Global variable - Declared above (comma before but|and|or is removed )
    # avoid connected values separated by splitting, e.g., ", but below 10%"
    text = re.sub(", ("+connect+") ", r' \1 ', text) # 

    # text is converted to lowercase
    return text.lower()

"mmol li+|mmol l|mol l|miu/ml minimum|miu/ml|centimetre of water|centimeter of water|cmH2O|cm H2O|x institutional upper limit of normal|x the upper limit of normal|x upper limits of normal|x upper limit of normal|x normal upper limit|x uln|the upper limit of normal|upper limits of normal|upper limit of normal|normal upper limit|uln|mm\\^3|mm3|kg2|ng ml|ng|ug|mmol|mol|percentiu|nmol|m\\^2|\\.m2|m\\(2\\)|kg m2|m2|kgm-2|in2|micromol|umol|mmhg|mm hg|millimeters of mercury|mm|hg|pmol|%|percent|uiu|iu|ul|ml min|ml|mg day|mg kg|g dl|mg dl|mg|dl|uln|gm|cm3|cc\\'s|cm|mcg|microns|rads|pg|um|torr|u|g|l|m|%|times|iuln|mcl|study|studies|cns|nyha|d|ptt|pt|inr|nsaid|copy|copies|iud|giga|F|C|v|fsh|wbc|plt|hgb|hpf|vwd|category|categories|oads|liquid stools|pack years|pack - years|kilograms|square meters|meters|platelets|millimoles|liters|kgs|grams|beats|lbs|miles|cells|degrees|drinks|proteins|msecs|patients|quadrants|readings|lesions|regimens|foots|cigarettes|crises|devices|doses|diameters|agents|units

In [ ]:
def split_text_inclusion_exclusion(otext):
    in_fea = 'inclusion criteria:|key inclusion criteria|inclusion criteria [^:#;\.]+:|inclusion:|(?<!(\w| ))inclusion criteria\W\W|inclusion for'
    ex_fea = 'exclusion criteria:|key exclusion criteria|exclusion criteria [^:#;\.]+:|exclusion:|(?<!(\w| ))exclusion criteria\W\W|exclusion for'
   
    in_text, ex_text = '', ''
    in_bool = True

    text = otext.lower()
    while text != '':
        if in_bool:
            n_pos = re.search('('+ex_fea+')',text)
            if n_pos is not None:
                in_text += text[0:n_pos.start()]
                text = text[n_pos.start():]
            else:
                in_text += text[0:]
                text = ''
        else:
            n_pos = re.search('('+in_fea+')',text)
            if n_pos is not None:
                ex_text += text[0:n_pos.start()]
                text = text[n_pos.start():]
            else:
                ex_text += text[0:]
                text = ''
        in_bool = False if in_bool else True
    
    sections_text =[]
    if in_text !='': sections_text.append(["Inclusion", in_text])
    if ex_text !='': sections_text.append(["Exclusion", ex_text])    
    return sections_text

In [5]:
import pandas as pd
ExampleData = pd.read_csv("data/example data diabetes_Type 1.csv",header=None)

In [9]:
pd.set_option('display.max_colwidth',-1)
ExampleData

0  \
0   NCT00004984   
1   NCT00005665   
2   NCT00021788   
3   NCT00021801   
4   NCT00034255   
5   NCT00042458   
6   NCT00042471   
7   NCT00042601   
8   NCT00046150   
9   NCT00063128   
10  NCT00067886   
11  NCT00071448   
12  NCT00073281   
13  NCT00095082   
14  NCT00095446   
15  NCT00097071   
16  NCT00097292   
17  NCT00100178   
18  NCT00105352   
19  NCT00107107   
20  NCT00108004   
21  NCT00109434   
22  NCT00117026   
23  NCT00117780   
24  NCT00118937   
25  NCT00118976   
26  NCT00119041   
27  NCT00127634   
28  NCT00129259   
29  NCT00130481   
..          ...   
70  NCT00252720   
71  NCT00252733   
72  NCT00254501   
73  NCT00260234   
74  NCT00265473   
75  NCT00271284   
76  NCT00272090   
77  NCT00273286   
78  NCT00276250   
79  NCT00276393   
80  NCT00278980   
81  NCT00279305   
82  NCT00279318   
83  NCT00283218   
84  NCT00284232   
85  NCT00285194   
86  NCT00285233   
87  NCT00286624   
88  NCT00286962   
89  NCT00290043   
90  NCT00290979   
91  NCT00291772   
92  NCT00297401   
93  NCT00297583   
94  NCT00297635   
95  NCT00298740   
96  NCT00303134   
97  NCT00304538   
98  NCT00305344   
99  NCT00306098   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   